In [1]:
%%capture
%pip install transformers
%pip install datasets loralib sentencepiece 
%pip install bitsandbytes accelerate
%pip install langchain
%pip install protobuf

# Loading Alpaca 7B

In [2]:
from transformers import CodeLlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

import torch

/home/ma-user/anaconda3/envs/py3.8/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tokenizer = CodeLlamaTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")

base_model = LlamaForCausalLM.from_pretrained(
    "codellama/CodeLlama-7b-hf",
    # load_in_8bit=True,
    device_map='auto',
    local_files_only=True
)

Loading checkpoint shards: 100%|██████████| 2/2 [01:24<00:00, 42.38s/it]


In [4]:
pipe = pipeline(
    "text-generation",
    model=base_model,
    do_sample=True,
    tokenizer=tokenizer, 
    max_length=4096,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.2
)

local_llm = HuggingFacePipeline(pipeline=pipe)

# Setting up a Chat with memory

In [5]:

from DBObject import DBObject

#import DBObject

#import importlib
#importlib.reload(DBObject)

In [6]:
from langchain.utilities import SQLDatabase

def connect_fun(database_name: str) -> DBObject:
  conn = SQLDatabase.from_uri(f"sqlite:///{database_name}?mode=ro") # open in readonly mode
  return DBObject(db=conn, llm=local_llm)

In [7]:

def query_fun(question: str, tables: list[str], database_connection: DBObject) -> str:
  response = database_connection.make_query(question, tables)

  return response


In [8]:
import testing

testing.run_test('example-data/sql-murder-mystery', connect_fun, query_fun)



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question: How many persons in the system?
Example query: SELECT count(*) FROM person
Example result:
   count(*)
0     10011


KeyboardInterrupt: 